In [1]:
!pip install -q dwave-ocean-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 26.8 MB/s eta 0:00:00


In [2]:
!pip install cheche_pm

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
import time
from tqdm import tqdm
from cheche_pm import Project

In [6]:
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dwave.system.samplers import DWaveSampler
from dimod.binary.binary_quadratic_model import BinaryQuadraticModel
from dimod.vartypes import Vartype

In [7]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = '<ENTER YOUR DWAVE API KEY HERE>'

# Input data

In [ ]:
inst = '< Insert the instance name here>'

In [ ]:
project = Project.from_rangen_1_rcp_file(file_name = inst)

In [ ]:
data = project.produce_outputs_for_MILP()

In [ ]:
n,p,S,u,c,ph = tuple(data.values())

(R, J, T) = (range(len(c)), range(len(p)), range(ph))
(R, J, T)
M = sum(p)

In [ ]:
nbOfActivities = n
nbOfPeriods = M
nbOfRessources = len(c)
timeActivities = p
capaRessources = c
activityRessourceConsumption = u
pairsListPrecedence = S
annealsNumber = 1000

In [ ]:
lambda_equalityConst =  nbOfPeriods
lambda_precedenceConst =  nbOfPeriods
lambda_ressourceConst =  nbOfPeriods
bigInteger  = 999999

In [ ]:
bqm = BinaryQuadraticModel(Vartype.BINARY)

In [ ]:
x = [[f'x_{i}_{t}' for t in T] for i in J]

# Create the BQM

In [ ]:
bqm = BinaryQuadraticModel(Vartype.BINARY)

In [ ]:
x = [[f'x_{i}_{t}' for t in T] for i in J]

In [ ]:
for t in T:
  bqm.add_variable(x[nbOfActivities+1][t], t)

## Objective function

In [ ]:
for t in T:
  bqm.add_variable(x[nbOfActivities+1][t], t)

## Constraints

### C1: Each activity must have an start date


In [ ]:
for i in J:
  terms = x[i]
  terms = [(x,1) for x in terms]
  bqm.add_linear_equality_constraint(terms=terms,lagrange_multiplier=lambda_equalityConst,constant=-1)
  terms.clear

### C2: Precedence constraints

In [ ]:
for precedenceIndex in range(len(pairsListPrecedence)): # precedence constraints
    activityOne = pairsListPrecedence[precedenceIndex][0]
    activityTwo = pairsListPrecedence[precedenceIndex][1]
    timeActivityOne = timeActivities[activityOne]
    for p1 in range(nbOfPeriods):
        for p2 in range(nbOfPeriods):
            if (p1 + timeActivityOne > p2):
                nameVar1 = "x_" + str(activityOne) +  "_" + str(p1)
                nameVar2 = "x_" + str(activityTwo) +  "_" + str(p2)
                bqm.add_quadratic(nameVar1,nameVar2,lambda_precedenceConst) # venturelli constraint
                #print("x ", nameVar1, " * ", nameVar2, " = 0 ")

### C3: Resource constraints

In [ ]:
for r in R:
  for t in T:
    terms = [(f'x_{j}_{t2}',u[j][r]) for j in J for t2 in range(max(0, t - p[j] + 1), t + 1)]
    bqm.add_linear_inequality_constraint(terms=terms,
                                         lagrange_multiplier=lambda_ressourceConst,
                                         label='res_',
                                         constant = -c[r],
                                         lb = -bigInteger,
                                         ub = 0)
    terms.clear

In [ ]:
print(dict(bqm.linear))

print("\n")

print(dict(bqm.quadratic))

In [ ]:
len(dict(bqm.linear))

In [ ]:
bqm.to_qubo() # genera un diccionario

# Solving the BQM

In [ ]:
sampler = DWaveSampler(solver='Advantage_system6.3',endpoint= endpoint, token = token)

In [ ]:
print("Connected to sampler", sampler.solver.name)

In [ ]:
print("Maximum anneal-schedule points: {}".format(sampler.properties["max_anneal_schedule_points"]))
print("Annealing time range: {}".format(sampler.properties["annealing_time_range"]))

max_slope = 1.0/sampler.properties["annealing_time_range"][0]

print("Maximum slope allowed on this solver is {:.2f}.".format(max_slope))

Maximum anneal-schedule points: 12
Annealing time range: [0.5, 2000.0]
Maximum slope allowed on this solver is 2.00.


In [ ]:
sampler = EmbeddingComposite(sampler)
sampler_name = sampler.properties['child_properties']['chip_id']

In [ ]:
sampleset = sampler.sample(bqm, num_reads= annealsNumber)